In [1]:
%matplotlib widget

from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd

import h5py
import skimage as sk
import json

from bouter.utilities import reliability 
from skimage.filters import threshold_otsu
import xarray as xr

from matplotlib import  pyplot as plt
import seaborn as sns
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()

from lotr.pca import pca_and_phase, fictive_heading_and_fit, fit_phase_neurons
from lotr import LotrExperiment

import lotr.plotting as pltltr
COLS = pltltr.COLS

from scipy import signal

In [2]:
master =  Path(r"\\Funes\Shared\experiments\E0071_lotr\full_ring")
fish_list = list(master.glob("*/*_f*_clol"))
path = fish_list[5]
path = Path(r"\\funes\Shared\experiments\E0071_lotr\full_ring\210314_f1\210314_f1_natmov")

In [32]:
master_path =  Path(r"\\funes\Shared\experiments\E0040_motions_cardinal\v21\2p\gad1b")
files = list(master_path.glob("*_f*"))
path = files[0]

In [33]:
# load fish data 

traces = fl.load(path / "filtered_traces.h5", "/detr")
traces_full = fl.load(path / "filtered_traces.h5", "/detr")
selected = fl.load(path / "selected.h5")
traces_hdn = traces_full[:, selected]
traces_hdn = traces_hdn.T

suite2p_data = fl.load(path / "data_from_suite2p_unfiltered.h5")
roi_map = suite2p_data['rois_stack']
coords = suite2p_data['coords']
anatomy = suite2p_data['anatomy_stack']

df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
exp = LotrExperiment(path)
fs = int(exp.fn)
beh_df = exp.behavior_log

t_start_s = 50
t_lims = (t_start_s*exp.fn, exp.n_pts - 50)
t_slice = slice(*t_lims)

OSError: ``\\funes\Shared\experiments\E0040_motions_cardinal\v21\2p\gad1b\20221021_f6\filtered_traces.h5`` does not exist

In [19]:
# get xcorr function for all cells with one of the cells
selected_neuron = 0
t = (np.arange(0,  (len_rec*2)-1) - len_rec) / fs

n_neurons, len_rec = np.shape(traces_hdn)
xcorr_all = np.zeros((n_neurons, (len_rec*2)-1))
for i in range(n_neurons):
    xcorr_all[i] = signal.correlate(traces_hdn[selected_neuron], traces_hdn[i])

In [24]:
# plot xcorr function 
fig, ax = plt.subplots(1, 1)
ax.imshow(xcorr_all, extent=[t[0], t[-1], 0, 10000], cmap='coolwarm')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
sorted_xcorr = xcorr_all[xcorr_all[:, len_rec].argsort()]
sorted_xcorr_show = sorted_xcorr[:, len_rec - 500:len_rec+500]
fig1, ax1 = plt.subplots(1, 1)
ax1.imshow(sorted_xcorr_show, extent=[-500, 500, 0, 1000], cmap='coolwarm')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …